In [1]:
import numpy as np
import keras
import keras.backend as k
from keras.layers import Conv2D,MaxPooling2D,SpatialDropout2D,Flatten,Dropout,Dense
from keras.models import Sequential,load_model
from keras.optimizers import Adam
from keras.preprocessing import image
import cv2
import datetime


# UNCOMMENT THE FOLLOWING CODE TO TRAIN THE CNN FROM SCRATCH

# BUILDING MODEL TO CLASSIFY BETWEEN MASK AND NO MASK

model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D() )
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        r'C:\Users\Dell\3D Objects\train',
        target_size=(150,150),
        batch_size=16 ,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        r'C:\Users\Dell\3D Objects\test',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary')

model_saved=model.fit_generator(
        training_set,
        epochs=10,
        validation_data=test_set,

        )



C:\Users\Dell\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


C:\Users\Dell\AppData\Local\Temp\ipykernel_12660\2829891200.py:50: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_saved=model.fit_generator(


Epoch 1/10
83/83 [==============================] - 74s 856ms/step - loss: 0.2852 - accuracy: 0.8875 - val_loss: 0.1057 - val_accuracy: 0.9588
Epoch 2/10
83/83 [==============================] - 55s 658ms/step - loss: 0.1730 - accuracy: 0.9316 - val_loss: 0.3696 - val_accuracy: 0.8557
Epoch 3/10
83/83 [==============================] - 54s 647ms/step - loss: 0.1268 - accuracy: 0.9536 - val_loss: 0.0959 - val_accuracy: 0.9588
Epoch 4/10
83/83 [==============================] - 54s 646ms/step - loss: 0.0908 - accuracy: 0.9681 - val_loss: 0.0866 - val_accuracy: 0.9742
Epoch 5/10
83/83 [==============================] - 54s 648ms/step - loss: 0.1065 - accuracy: 0.9635 - val_loss: 0.1163 - val_accuracy: 0.9536
Epoch 6/10
83/83 [==============================] - 57s 689ms/step - loss: 0.0673 - accuracy: 0.9734 - val_loss: 0.0678 - val_accuracy: 0.9639
Epoch 7/10
83/83 [==============================] - 53s 643ms/step - loss: 0.0936 - accuracy: 0.9665 - val_loss: 0.2977 - val_accuracy: 0.9278

In [4]:
model.save('mymodel.h5',model_saved)

#To test for individual images

mymodel=load_model('mymodel.h5')
#test_image=image.load_img('C:/Users/Karan/Desktop/ML Datasets/Face Mask Detection/Dataset/test/without_mask/30.jpg',target_size=(150,150,3))
test_image=image.load_img(r'C:\Users\Dell\3D Objects\test\with_mask\1-with-mask.jpg',
                          target_size=(150,150,3))

test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
mymodel.predict(test_image)[0][0]


# IMPLEMENTING LIVE DETECTION OF FACE MASK

mymodel=load_model('mymodel.h5')

cap=cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
          
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 80ms/step


1/1 [==============================] - 0s 64ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 82ms/step
